In [83]:
from faker import Faker
import random
import pandas as pd
import numpy as np
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from base64 import b64encode, b64decode

# Initialize faker
fake = Faker()

class AESCipher:
    def __init__(self, key):
        self.key = key

    def encrypt(self, data):
        cipher = AES.new(self.key, AES.MODE_EAX)
        ciphertext, tag = cipher.encrypt_and_digest(data)
        return cipher.nonce, b64encode(ciphertext).decode('utf-8')

# Define a function to generate a customer
def generate_customer_profile():
    profile = fake.profile()
    return {
        "name": profile['name'],
        "address": fake.address().replace('\n', ', '),
        "email": profile['mail'],
        "phone_number": fake.phone_number(),
        "social_security_number": fake.ssn(),
        "job": profile['job'],
        "company": profile['company'],
        "residence": fake.address().replace('\n', ', '),
        "current_location": fake.address().replace('\n', ', '),
        "website": profile['website'][0] if profile['website'] else None,
        "username": profile['username'],
        "age": fake.random_int(min=18, max=80, step=1),
        "gender" : profile['sex'],
        "Grades": fake.random_int(min=0, max=100, step=1),
        "preferred_language": fake.language_code(),
        "preferred_currency": fake.currency_code(),
        "job_title": profile['job'],
        "national_id": fake.random_int(min=1000000000, max=9999999999, step=1),
        "height": fake.random_int(min=100, max=200, step=1),
        "weight": fake.random_int(min=100, max=200, step=1),
    }

# Define a function to generate a transaction
def generate_transaction(name):
    return {
        "name": name,
        "amount": f"${fake.random_int(min=1, max=1000, step=1):.2f}",
        "date": fake.date_time_between(start_date='-1y', end_date='now', tzinfo=None),
    }

# Generate customers
customers = [generate_customer_profile() for _ in range(10)]
customers_df = pd.DataFrame(customers)



In [84]:

# Generate transactions for each customer randomly picking a customer by name
transactions = [generate_transaction(random.choice(customers)["name"]) for _ in range(20)]
transactions_df = pd.DataFrame(transactions)
transactions_df.head()

,name,amount,date
0,Eric Gomez,$635.00,2022-11-09 00:45:51
1,Lauren Moss,$625.00,2022-11-16 15:34:36
2,Tina Boyd,$757.00,2022-06-29 02:39:05
3,Jeffrey Morrison,$584.00,2023-04-07 20:55:50
4,Tina Boyd,$715.00,2022-07-29 21:56:29


In [85]:

# Encrypt the data in the CRM
password = get_random_bytes(16)  # In a real system, never hard-code the password
cipher = AESCipher(password)


In [86]:

encrypted_df = pd.DataFrame()
encryption_key = pd.DataFrame()
for column in customers_df.columns:
    encrypted_column = []
    key_column = []
    for data in customers_df[column]:
        iv, ct = cipher.encrypt(str(data).encode('utf-8'))
        encrypted_column.append(ct)
        key_column.append(iv)
    encrypted_df[column] = encrypted_column
    encryption_key[column] = key_column

print("password: ", password)
# save the encrypted data (first five rows as a markdown table)
md_table = encrypted_df.head().to_markdown()
# save md table to file
with open("encrypted_data.md", "w") as f:
    f.write(md_table)

# Read the README.md file
readme_text = ""
with open("README.md", "r") as f:
    readme_text = f.read()
# open the README.md file and add the encrypted data at the end
with open("README.md", "w") as f:
    # read what is currently in the README.md file
    readme = readme_text
    # add the encrypted data
    readme += "\n\n## Encrypted Data\n\n"
    readme += md_table
    # write the new README.md file
    f.write(readme)

# Print the encrypted data
encrypted_df.head()

password:  b'\n\xd4i\xb6S\x13\xf88\x13\xb1\x83\xbd\xa1[bX'


,name,address,email,phone_number,social_security_number,job,company,residence,current_location,website,username,age,gender,Grades,preferred_language,preferred_currency,job_title,national_id,height,weight
0,PLfNe3cXxPTSXMiSXyz5Ew==,ocrRxgQcrXNA3TQjgB+Y9m8ZglUrLOxTpBMX5AmLFLaARW...,qLbp1AdF4TVDY1Te/zcrzUTL,SDumXTWBkYtt0w==,UGz/tmEZtM+x9L0=,Qppj0LCyZ1dBmbyAUrHsnMjec/dOiZr1i9ZXpg==,lINNyM3F9KdgE39Hj2mNo3Bxa2UsfWm3,YPZklsWHk665uHVY7FBrNY6nQNy5s59t/y8JyoluJ7a99m...,d84sD76MoiaZ5y1BkIIOP3ACp1rf9oLnOYn7NnXaInfN2U...,IxMNcm59NKs9D1ymSyjzI9stVB3US/c=,ZsEUvafYr0y/,qQo=,qQ==,sY0=,WlU=,r8ej,LBq9UPwe130M7uvDduPk+04nrtVS2DgODW0b7A==,XcvqaFI2Tvm/UQ==,UDtT,DjKu
1,z+ekBOlx7AI4b9SpDOAIFg==,5q66YJrP4WYjIYXfxWneIVn/IQnR+zFd4FkZuqxE7KYai7...,isk8R5nhyNCmEPkZj2YGuWHBUw==,pTfufcBF004x9sykC89xh59YicM=,MEM36KJ399oDAVI=,S59rcs1ycwxLjg/Ue/E=,kWCpRtuKzdtw3g==,7nsVocZ459TfDQJfMhuCVuvjg1jJEN2EbS8jcoSGvw/fTV...,1PikXX1BWetvve6dly+4tK2WbcenFX5l4Clvgu+DrsWnVG...,f51N0BhxNV8JaiSx4sgPl22wgegJpPTj,i7J4sHq87A==,GSA=,IA==,18s=,ugBp,lOe6,MQXo1mK2h5sR+5DaC2c=,+fll5dTfVGj69Q==,a3Fi,wjIl
2,65jiHKI2Ka9KLxE=,1rWxCSUDouidFyXS88ZzIWkQhXljgQwpzLiKT01OfDuXoH...,gMv7iJjMiYtZwe7Q1PSz7MA=,efy+UT2ZPdZCAUy3,AIjFvmwkBiQubRQ=,/OgZ+g8KcJE2YyhDQx4WR8elOCKP2sT3qdXM4AgYmlC2Xp...,e+Si7xW0ouW958Gf,ZetPJq8jWUAVzDGtepYivXr+9si5PDYKGSyaqvimkh5rGs...,n2dqP0y0vDg0P8VdVCHoxz5BEDG7XgWZvam8PjVpzVGii2...,fcR1H3RPKDAKH9eliG30znrIxxiFmzU=,FxRZKqgqtHQ=,VY4=,Ow==,vik=,nR1u,yL4J,mh/7A925Q0jLH05JdvqODt+UTiXDYdG2T2PfgtqQcvSjXG...,Ir5EF7sFzcrBhg==,5XlV,2RfN
3,5GdPb1qTP1qBBOT/5mWzbA==,48g2KRLhxbemgou2U62EgyENTBM2ALmUBJv92v9v+kfVC6...,Syc8Jpp0H5jvZqAal1MxmzDwjwMl,M3NnPfIBjs13SRHqTCAM9RPp,2TJ4ASGiwt4/038=,g6ZpSMKQyxAA4UEQV7oq,RtwUl0SZYI5HbUAss352L+GKG3uuywhr7g4=,UnNNUBd2QxVMs9boSmuhYmHidLZ1kDke9FoaNNEAUd53n3...,KXi5CMQmEPVEOwTkzUhEx1hFNBlfU3x3G8yWpefwzcVehq...,WqW1ASbu38Q9LbatrAJWX3pJqWzwGFKt/TaZdw==,jQPjEMuyhjcigVQ=,XUc=,5g==,Cg==,5uI=,c9X5,EHaSkPxGLA0pYZLnAJDd,LeqwoWSda4lt8A==,H4o+,vtnj
4,RBJx5CRz+MvVjU6Z,GQozrST77wJTY7KrwOf1/fC5nGF0bD5Q/t7SvYw/n3/tYr...,gtkRhXMHn9IKcRb/GxwnP6GW9rmG1A2bU9sq,8uY1epTnjU3mzg==,9RYCBeNjgrPKEx0=,/NBeiJ34PLG+xwUb,+CyU6UtndjHySJwBFvE=,DbhndlQrhYDkDWx2UrAFrvwx7ZRjORIibHP/utryv8HI+e...,QKhVlr/7yBcyKZsUsKoZgkdo4kHisi10j8XNl1DgPVpH+w...,ZFkFil3Pjq/TKSWyoE7HwwhNtRDi,3gCOO2Sehj1YengXOg==,BWU=,gw==,yHU=,1hw=,/bdo,lItkr3eWwSNvBra5,fl5SaypVdTKbFg==,pQ23,Pt8H
